# Exercise 02 : Join

In [1]:
import pandas as pd 
import sqlite3

## create a connection to the database using the library sqlite3

In [2]:
connection = sqlite3.connect('../../data/checking-logs.sqlite')
cursor = connection.cursor()

## create a new table datamart in the database by joining the tables pageviews and checker using only one query
- the table should have the following columns: uid, labname, first_commit_ts, first_view_ts
- first_commit_ts is just a new name of the column timestamp from the checker table, it shows the first commit from a particular lab and from a particular user
- first_view_ts is the first visit of a user to the table pageviews, timestamp when a user visited the newsfeed
- status = ’ready’ should still be a filter
- numTrials = 1 should still be a filter
- labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’
- the table should contain only the users (uids with user_*) and not the admins
- first_commit_ts and first_view_ts should be parsed as datetime64[ns]

In [3]:
drop_table_query = "DROP TABLE IF EXISTS datamart;"
cursor.execute(drop_table_query)
connection.commit()

In [4]:
#checkers = pd.io.sql.read_sql("SELECT uid, labname, timestamp FROM checker WHERE status = 'ready' AND (uid NOT LIKE 'admin_%') AND numTrials = 1 AND labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') ORDER BY uid ASC", connection)
#checkers

In [5]:
PAGEVIEWS = pd.io.sql.read_sql("SELECT uid, MIN(datetime) AS datetime FROM pageviews WHERE (uid LIKE 'user_%') GROUP BY uid", connection)
PAGEVIEWS

,uid,datetime
0,user_1,2020-04-26 21:53:59.624136
1,user_10,2020-04-18 12:19:50.182714
2,user_14,2020-04-18 10:53:52.623447
3,user_17,2020-04-18 10:56:55.833899
4,user_18,2020-04-26 22:49:29.243278
5,user_19,2020-04-21 20:30:38.034966
6,user_21,2020-04-22 22:40:36.824081
7,user_25,2020-05-09 23:54:54.260791
8,user_28,2020-05-10 21:07:50.350946
9,user_3,2020-05-08 10:53:47.123832


In [6]:
insert_data_query = """
CREATE TABLE datamart AS
SELECT
    c.uid,
    c.labname,
    MIN(c.timestamp) AS first_commit_ts,
    MIN(p.datetime) AS first_view_ts
FROM
    checker c
LEFT JOIN
    pageviews p ON c.uid = p.uid
    WHERE c.status = 'ready' 
    AND c.numTrials = 1 
    AND c.labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND (c.uid LIKE 'user_%')

GROUP BY 
    c.uid, 
    c.labname

"""
# OR c.uid IS NULL
cursor.execute(insert_data_query)
connection.commit()


In [7]:
datamart = pd.io.sql.read_sql("SELECT * FROM datamart", connection, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [8]:
datamart.head(20)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## using Pandas methods, create two dataframes: test and control
- test should have the users that have the values in first_view_ts
- control should have the users that have missing values in first_view_ts

In [9]:
#test_df = (datamart.dropna(subset=['first_view_ts']).reset_index(drop=True))
test_df = datamart.dropna(subset=['first_view_ts'])
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [10]:
control_df = datamart[datamart['first_view_ts'].isna()]
control_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


- replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis
- save both tables into the database, you will use them in the next exercises

In [11]:
average_test_first_view_ts = test_df['first_view_ts'].mean()
average_test_first_view_ts

Timestamp('2020-04-27 00:40:05.761783552')

In [12]:
control_df = control_df.copy()
control_df['first_view_ts'] = control_df['first_view_ts'].fillna(average_test_first_view_ts)
#control_df.reset_index(drop=True, inplace=True)
control_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [13]:
test_df.to_sql('test', connection, if_exists='replace', index=False)

59

In [14]:
control_df.to_sql('control', connection, if_exists='replace', index=False)

81

In [15]:
connection.close()